# SMART POWER MONITERING SYSTEM

In Our Project XILINX PYNQ Z2 is used as a Main Processing Unit also used as a database which Stores Data and applies Machine Learning Algorithm for Early Prediction of Errors in the Power Grid Prior to the fault. It maintains Grid Database and Sends Reports to the Authority periodically. This Report is composed of Grid Condition,Transformer Health,life Expectency,Voltage and Current readings,faulty Pole Tracking data,short circuit detection,Power Theft info,Transformer switching. 

Features of Pynq Z2 board in out project:
>.Data base  (_Stores all Data Related to the Grid_)

>.Early Fault Prediction   (_It predicts the Error in the grid Prior to the fault to occue _)


>.Grid Network Report

>.Hardware Interface

>.Problem Classification

>.Seasonal Grid Health Prediction

>.Anti Power Theft
>

Importing Base Overlays for accessing GPIO pins on Pynq  Board . 

In [2]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

Libraries for including features like Email, Database,date ,time,seasons,Arduino Gpio Control.

In [3]:
from pynq.lib.arduino import arduino_io
import time
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from csv import reader
from sys import exit
from math import sqrt
from operator import itemgetter
import csv
from csv import reader
from sys import exit
from math import sqrt
from operator import itemgetter
import pytz
import random
from datetime import datetime
from dateutil.tz import *

We are Configuring Arduino I/O ports for Controlling Power unit and to Setup all default Data

In [4]:
led = arduino_io.Arduino_IO(base.ARDUINO, 13, 'out')
d1=arduino_io.Arduino_IO(base.ARDUINO,2,'in')
d2=arduino_io.Arduino_IO(base.ARDUINO,3,'in')
f=arduino_io.Arduino_IO(base.ARDUINO,4,'in')
sc=arduino_io.Arduino_IO(base.ARDUINO,5,'in')
mail=arduino_io.Arduino_IO(base.ARDUINO,6,'in')
flag=0
season=0
month=0
load=0
health='_'
problem_time='_'

These are the functions used to set up Month,Date, time,seasons,Email and to take Real time data as Sensor input(current,voltage,Fuse,Load..) .

In [5]:
breakdowns=0
def gettime():
    now = datetime.now()
                                              # assuming now contains a timezone aware datetime
    zone = pytz.timezone('Asia/Kolkata')
    TIME_now = now.astimezone(zone)
    #print('time =',(TIME_now).time())
    #print('date =',(TIME_now).date())        #datetime.datetime.now().date()
    nela=(TIME_now).date().month
    #print('month =',nela)
    return TIME_now
def predictmonth():   
    maheen=(gettime().month)
    if(maheen in range(3,6)):
        seas=3                #summer
    elif(maheen in range(7,10)):
        seas=2                #rainy
    else:
        seas=1               #winter
    return seas 
def seasons():                 # sesons 
    season=predictmonth()
    #print(season)
    return season
def temperature():          # Transformer temperature readings using randint in place of sensors
    if(season in range(1,2)):
        temp=random.randint(50,80)
        #print(temp)
        return temp 
    else:
        temp=random.randint(75,90)
        #print(temp)
        return temp  
def outload():             #using randint in place of sensors 
    if(season in range(1,2)):
        load=random.randint(10,50)
        #print(load)
        return load
    else:
        load=random.randint(40,80)
        #print(load)
        return load    
def fusedowns():            #using randint in place of sensors
    breakdowns=0   
                           #breakdowns=breakdowns+1 
    return breakdowns 
    
        
def pole():      # taking Pole data from Pole Transmitters
    p1=d1.read()
    #print('p1=',p1)
    p2=d2.read()
    #print('p2=',p2)
    polenumber=p1*2+p2*1
    if(polenumber>=1):
        problem_time=gettime()
    return polenumber
def addtestdata():     # Giving test data to Machine Leanrnig Algorithm
    with open('powertest.csv', mode='w') as employee_file:
        employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        print('season=',seasons())
        employee_writer.writerow([seasons(),temperature(),outload(),fusedowns()])
def email():             # For sending Email to The Authority
    email_user = 'Your Email id'
    email_password = 'your Pswd'
    email_send = 'To mail id'
    subject = 'SMART POWER MONITORING SYSTEM Powered by PYNQ Z2'
    msg = MIMEMultipart()
    msg['From'] = email_user
    msg['To'] = email_send
    msg['Subject'] = subject
    body = 'Hi there, sending this email from Python!'
    msg.attach(MIMEText(body,'plain'))
    filename='smartpower_report.txt'
    attachment  =open(filename,'rb')
    part = MIMEBase('application','octet-stream')
    part.set_payload((attachment).read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition',"attachment; filename= "+filename)
    msg.attach(part)
    text = msg.as_string()
    server = smtplib.SMTP('smtp.gmail.com',587)
    server.starttls()
    server.login(email_user,email_password)
    server.sendmail(email_user,email_send,text)
    print('mail sent')
    server.quit()

This Algorithm is used for Electric grid health prediction and to add data into csv file .

In [6]:
def load_data_set(filename):
    try:
        with open(filename, newline='') as iris:
            return list(reader(iris, delimiter=','))
    except FileNotFoundError as e:
        raise e


def convert_to_float(data_set, mode):
    new_set = []
    try:
        if mode == 'training':
            for data in data_set:
                new_set.append([int(x) for x in data[:len(data)-1]] + [data[len(data)-1]])

        elif mode == 'test':
            for data in data_set:
                new_set.append([int(x) for x in data])

        else:
            print('Invalid mode, program will exit.')
            exit()

        return new_set

    except ValueError as v:
        print(v)
        print('Invalid data set format, program will exit.')
        exit()


def get_classes(training_set):
    return list(set([c[-1] for c in training_set]))


def find_neighbors(distances, k):
    return distances[0:k]


def find_response(neighbors, classes):
    votes = [0] * len(classes)

    for instance in neighbors:
        for ctr, c in enumerate(classes):
            if instance[-2] == c:
                votes[ctr] += 1

    return max(enumerate(votes), key=itemgetter(1))


def knn(training_set, test_set, k):
    distances = []
    dist = 0
    limit = len(training_set[0]) - 1

    # generate response classes from training data
    classes = get_classes(training_set)

    try:
        for test_instance in test_set:
            for row in training_set:
                for x, y in zip(row[:limit], test_instance):
                    dist += (x-y) * (x-y)
                distances.append(row + [sqrt(dist)])
                dist = 0

            distances.sort(key=itemgetter(len(distances[0])-1))

            # find k nearest neighbors
            neighbors = find_neighbors(distances, k)

            # get the class with maximum votes
            index, value = find_response(neighbors, classes)
            outputclass=classes[index]
            # Display prediction
            print('The predicted class for sample [Season ,core Temp ,power Load ,blown Fuses] : Health of Transformer' )
            print('The predicted class for sample ' + str(test_instance) + ' is : ' + classes[index])
            print('Number of votes : ' + str(value) + ' out of ' + str(k))
            if (str(value)==str(k)):
                with open('powerdataset.csv','r') as csvinput:
                    with open('powerdataset.csv', 'a') as csvoutput:
                        writer = csv.writer(csvoutput)
                        berry=classes[index]
                        newrow=test_instance+[str(berry)]                        
                        writer.writerow(newrow)
            # empty the distance list
            distances.clear()
            

    except Exception as e:
        print(e)
    finally:
        return outputclass

Invoking the grid health prediction algorithm in the code which takes data from the code and predicts the Health and Power theft based on sensor data.

In [7]:
def ML():
    try:
        # get value of k
        k = 5 #int(input('Enter the value of k : '))
        # load the training and test data set
        training_file ='powerdataset.csv' #input('Enter name of training data file : ')
        test_file = 'powertest.csv'#input('Enter name of test data file : ')
        training_set = convert_to_float(load_data_set(training_file), 'training')
        test_set = convert_to_float(load_data_set(test_file), 'test')

        if not training_set:
            print('Empty training set')

        elif not test_set:
            print('Empty test set')

        elif k > len(training_set):
            print('Expected number of neighbors is higher than number of training data instances')

        else:
            health=knn(training_set, test_set, k)

    except ValueError as v:
        print(v)

    except FileNotFoundError:
        print('File not found')
    finally:
        return health

A Report mail will be sent to Power management Regarding the Grid Network reports periodically
with format

 ---------------------------------------------------------
###     SMART POWER MONITORING powered by SPMOS            
 ---------------------------------------------------------
>Transformer Service No: 1#008337hgj6745
<br>
>AREA : Vijayawada ,colony: Benz Circle ,Line no 21, Street 52
<br>
>Consumed UNITS :4800
<br>
>Health of Transformer: good
<br>
>Life Expectency :28
<br>
>Future Faults :no error
<br>
>Grid Condition :good
<br>
>Short circuits :0
<br>
>Problem at Pole :No problems
<br>
>Power Theft :0. 

In [8]:
def texttomail(swasth):
    file1 = open("smartpower_report.txt","w") 
    #load=130
    health=swasth
    if(health=='normal'):
        life=random.randint(10,20)
    elif(health=='good'):
        life=random.randint(20,30)
    else:
        life=random.randint(3,10)
    if(pole()>=1):
        mypole=pole()
        problem_time=gettime()
        print('problem at pole happened at \n')
        print(problem_time)
    else:
        mypole=0
        problem_time='_'
    
    L = [" ---------------------------------------------------------\n",
        "|     SMART POWER MONITORING powered by SPMOS             |\n ",
         "---------------------------------------------------------\n",
         "|Transformer 1","\n","|AREA : ponnur Line no 21, Street 52\n",
         "|Power UNITS :",str(outload()),"\n",
         "|Health of Transformer:",str(health),"\n",
         "|Life Expectency :",str(life),"\n",
         "|Future Faults :",str('no error'),"\n",
         "|Grid Condition :",str(health),"\n",
         "|Short circuits :",str(fusedowns()),"\n",
         "|Problem at Pole :",str(mypole),"at time :",str(problem_time),"\n",#type pole data and time
         "|Power Theft :",str(fusedowns()),"\n",
        ]  
    print('problematic pole =',pole())
    # \n is placed to indicate EOL (End of Line) 
    file1.write("Hello \n") 
    file1.writelines(L) 
    file1.close() #to change file access modes 

Continuosly monitoring the above factors inside this loop.

In [11]:
def loop():
    while(1):
        addtestdata() # reads testdata
        time.sleep(1)
        swasth=ML()           #predict test data
        time.sleep(1)
        texttomail(swasth)  #generates a text document to be sent by mail
        time.sleep(1)
        if mail.read()==1:  #if mail summons
            email()  #sending mail to admin
        time.sleep(50)    
  



Driving program to run the entire code.

Data input From Sensors<br> 
>Seasons <br> Voltage <br> Current <br> Transformer Core Temperature <br> Grid Load <br> Blown Fuses : <br><br><br> Gives Health of Transformer

In [13]:
def main():
    loop()

if __name__ == '__main__':
    main()


season= 1
The predicted class for sample [Season ,core Temp ,power Load ,blown Fuses] : Health of Transformer
The predicted class for sample [1, 80, 52, 0] is : normal
Number of votes : 4 out of 5
problematic pole = 0


KeyboardInterrupt: 